In [6]:
from MCForecastTools import MCSimulation
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import requests
import json
import chart_studio
import chart_studio.plotly as cpx
from dotenv import load_dotenv
import os
import warnings 
warnings.filterwarnings('ignore') 

In [3]:
load_dotenv()
apikey = os.getenv("APIKEY")

In [2]:
class MonteCarlo:
    def __init__(self, ticker, timeperiod, num_simulations=500, initial_investment=10000):
        self.ticker = ticker
        self.timeperiod = timeperiod
        self.num_simulations = num_simulations
        self.initial_investment = initial_investment
        
    def get_ticker_data(self):
        API_KEY = apikey
        res = requests.get('https://api.glassnode.com/v1/metrics/market/price_usd_ohlc', params={'a':self.ticker,
                                                                                                 'api_key':API_KEY})
        data = json.loads(res.text)
        final = []
        for dictionary in data:
            result = {}
            result['Date'] = datetime.fromtimestamp(dictionary['t']).date()
            result['Open'] = dictionary['o']['o']
            result['Close'] = dictionary['o']['c']
            result['High'] = dictionary['o']['h']
            result['Low'] = dictionary['o']['l']
            final.append(result)
        self.dataframe = pd.DataFrame(final)
        self.dataframe.set_index('Date', inplace=True)
        self.dataframe.columns = ['open', 'high', 'low', 'close']
        self.dataframe.columns = pd.MultiIndex.from_product([[self.ticker], self.dataframe.columns])


    def run_monte_carlo_simulations(self):
        self.mc_obj = MCSimulation(portfolio_data = self.dataframe, num_simulation = self.num_simulations,
                                   num_trading_days = 252*self.timeperiod)
        self.mc_obj.calc_cumulative_return()
        
    def plot_monte_carlo_simulations(self):
#         username = "Anurag1097"
#         api_key = "TKDao0r2KPRZu1xk6O0e"
#         chart_studio.tools.set_credentials_file(username=username, api_key=api_key)
        plotly_fig = px.line(self.mc_obj.simulated_return, x=self.mc_obj.simulated_return.index,
                             y=self.mc_obj.simulated_return.columns, height=600, width=1000,
                             labels={'value':'Percentage Return', 'variable':'Simulation',
                                     'index':'Number of Trading Days'})
        plotly_fig.show()
#         cpx.plot(plotly_fig, filname = f"MonteCarlo Line Chart for {self.ticker}", auto_open=False)
    
    def get_summary_statistics(self):
        self.summary_analysis = self.mc_obj.summarize_cumulative_return()
        
    def get_confidence_interval(self):
        upper_fence = round(self.summary_analysis[8]*self.initial_investment, 2)
        lower_fence = round(self.summary_analysis[9]*self.initial_investment, 2)
        median_return = round(self.summary_analysis[5]*self.initial_investment, 2)
        mean_return = round(self.summary_analysis[1]*self.initial_investment, 2)
        print(f"There is a 95% chance that an initial investment of ${self.initial_investment} in the portfolio"
        f" over the next {self.timeperiod} years could end up within in the range of"
        f" ${'{:,.2f}'.format(lower_fence)} and ${'{:,.2f}'.format(upper_fence)}, with mean value as ${'{:,.2f}'.format(mean_return)} and"
        f" median value as ${'{:,.2f}'.format(median_return)}")

In [3]:
obj = MonteCarlo("ETH", 10, 250000)
obj.get_ticker_data()
obj.run_monte_carlo_simulations()
obj.get_summary_statistics()
obj.get_confidence_interval()
#obj.plot_monte_carlo_simulations()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si